<a href="https://colab.research.google.com/github/mvasq723/Class/blob/main/Max_Vasquez_Spark_Prophet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Running Spark in Colab For Time Series
# BASED ON CODE FROM Anik (2019) AND Thomas (2020)

%matplotlib inline
!apt update

import timeit


Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:8 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:14 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:15 http://archive.ubuntu.co

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://www-us.apache.org/dist/spark/spark-2.4.7/spark-2.4.7-bin-hadoop2.7.tgz
!tar xf spark-2.4.7-bin-hadoop2.7.tgz
!pip install -q findspark

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.7-bin-hadoop2.7"

In [ ]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [ ]:
from pyspark.sql.functions import pandas_udf, PandasUDFType
from pyspark.sql.types import StructType,StructField,StringType,LongType,DoubleType,IntegerType,FloatType,DateType,TimestampType

import statsmodels.tsa.api as sm
import numpy as np
import pandas as pd
from fbprophet import Prophet
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

In [ ]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression

In [ ]:
# Load Data to realestate
realestate = pd.read_csv("http://files.zillowstatic.com/research/public/Zip/Zip_Zhvi_SingleFamilyResidence.csv")
print(realestate)
print("\n\n\n-----Code executed succesfully-----")

       RegionID  SizeRank  RegionName RegionType StateName  ... 2019-11-30  \
0         61639         0       10025        Zip        NY  ...  1375725.0   
1         84654         1       60657        Zip        IL  ...   973104.0   
2         61637         2       10023        Zip        NY  ...  1480426.0   
3         91982         3       77494        Zip        TX  ...   336083.0   
4         84616         4       60614        Zip        IL  ...  1201182.0   
...         ...       ...         ...        ...       ...  ...        ...   
30459     58111     35187         802        Zip        UT  ...   130717.0   
30460     58115     35187         820        Zip        LA  ...   101363.0   
30461     58117     35187         822        Zip        LA  ...   175176.0   
30462     58121     35187         831        Zip        AL  ...    74273.0   
30463     58125     35187         851        Zip        CO  ...   450047.0   

      2019-12-31 2020-01-31 2020-02-29  2020-03-31  
0      137

In [ ]:
pd.set_option('display.max_rows', 10)
pd.set_option('display.max_columns', 10)
print("\n\n\n-----Code executed succesfully-----")




-----Code executed succesfully-----


In [ ]:
# Change realestate data to dataframe
realestate=pd.DataFrame(realestate)
print("\n\n\n-----Code executed succesfully-----")




-----Code executed succesfully-----


In [ ]:
##################################
#Data Munging (Original)
##################################
# Identify length of rows and columns
norows=len(realestate.RegionID)
nocol=len(realestate.columns)-9                                               # -9 because the 1st 9 columns not being transposed

# Create new data frame with columns not being being transposed
realestatelong=realestate[['RegionID','RegionName','City','State','Metro','CountyName','SizeRank']].astype(str)
realestatelong=pd.concat([realestatelong]*nocol, ignore_index=True)

# Add new columns
realestatelong['date']=''
realestatelong['value']=''

#first row
x=0
#last row
y=30464
#Starting column
z=9

# Loop
#--------------------------
for i in range(nocol):

  realestatelong.date[x:y]=realestate.columns[z]  
  realestatelong.value[x:y]=realestate.iloc[:,z]
 
  x=x+norows
  y=y+norows
  z=z+1

# end of loop ----------------------------
print("\n\n\n-----Code executed succesfully-----")





-----Code executed succesfully-----


In [ ]:
#sort data by Region Name and date
realestatelong=realestatelong.sort_values(["RegionName","date"], ascending=(True,True))
print(realestatelong)
print("\n\n\n-----Code executed succesfully-----")

        RegionID RegionName      City State                        Metro  \
1579       61615      10001  New York    NY  New York-Newark-Jersey City   
32043      61615      10001  New York    NY  New York-Newark-Jersey City   
62507      61615      10001  New York    NY  New York-Newark-Jersey City   
92971      61615      10001  New York    NY  New York-Newark-Jersey City   
123435     61615      10001  New York    NY  New York-Newark-Jersey City   
...          ...        ...       ...   ...                          ...   
8729552   100489      99929  Wrangell    AK                          nan   
8760016   100489      99929  Wrangell    AK                          nan   
8790480   100489      99929  Wrangell    AK                          nan   
8820944   100489      99929  Wrangell    AK                          nan   
8851408   100489      99929  Wrangell    AK                          nan   

               CountyName SizeRank        date   value  
1579      New York County     

In [ ]:
#Drop Unwanted columns
re_prophet=realestatelong.drop(columns = ['RegionID', 'State', 'Metro', 'CountyName', 'SizeRank']) #drop unenecessary columns
print("\n\n\n-----Code executed succesfully-----")




-----Code executed succesfully-----


In [ ]:
#Change date column from object to datetime
re_prophet.date=pd.to_datetime(re_prophet['date'], format='%Y-%m-%d')

#Change value from object to float
re_prophet.value=re_prophet.value.astype(float)

#Check for Nan
print("\nFeatures and data types : \n",re_prophet.dtypes)
print("\nMissing values : \n" , re_prophet.isnull().any())
print("\nUnique values : \n" , re_prophet.nunique())
print("\n\n\n-----Code executed succesfully-----")


Features and data types : 
 RegionName            object
City                  object
date          datetime64[ns]
value                float64
dtype: object

Missing values : 
 RegionName    False
City          False
date          False
value          True
dtype: bool

Unique values : 
 RegionName     30464
City           14862
date             291
value         727935
dtype: int64



-----Code executed succesfully-----


In [ ]:
##########partition test
#re_prophet
#x=30464/7
#print(x)
#y=x*291
#print(y)
#df.iloc[:1000,:]
#print(re_prophet.iloc[:1266432,:] ) 
#print(re_prophet.iloc[1266432:2532864,:] )
#print(re_prophet.iloc[2532864:3799296,:] )
#print(re_prophet.iloc[3799296:5065728,:] )
#print(re_prophet.iloc[5065728:6332160,:] )
#print(re_prophet.iloc[6332160:7598592,:] )
#print(re_prophet.iloc[7598592:,:] )


re_prophet0=re_prophet.iloc[:291,:]
re_prophet1=re_prophet.iloc[:1266432,:]
re_prophet2=re_prophet.iloc[1266432:2532864,:]
re_prophet3=re_prophet.iloc[2532864:3799296,:]
re_prophet4=re_prophet.iloc[3799296:5065728,:]
re_prophet5=re_prophet.iloc[5065728:6332160,:]
re_prophet6=re_prophet.iloc[6332160:7598592,:]
re_prophet7=re_prophet.iloc[7598592:,:]
print(re_prophet0)

        RegionName      City       date      value
1579         10001  New York 1996-01-31   279614.0
32043        10001  New York 1996-02-29   284673.0
62507        10001  New York 1996-03-31   285884.0
92971        10001  New York 1996-04-30   292432.0
123435       10001  New York 1996-05-31   295610.0
...            ...       ...        ...        ...
8714283      10001  New York 2019-11-30  1160163.0
8744747      10001  New York 2019-12-31  1153438.0
8775211      10001  New York 2020-01-31  1154711.0
8805675      10001  New York 2020-02-29  1151760.0
8836139      10001  New York 2020-03-31  1149756.0

[291 rows x 4 columns]


In [ ]:
#re_prophet.to_csv('re_prophet.csv')

In [ ]:
schema = StructType([StructField('RegionName', StringType(), True),
                     StructField('City', StringType(), True),
                     StructField('date', TimestampType(), True),
                     StructField('value', DoubleType(), True)])
print("\n\n\n-----Code executed succesfully-----")




-----Code executed succesfully-----


In [ ]:
#create a spark dataframe
sre_prophet=spark.createDataFrame(re_prophet0,schema)
print("\n\n\n-----Code executed succesfully-----")




-----Code executed succesfully-----


In [ ]:
sre_prophet.show(5)
print("\n\n\n-----Code executed succesfully-----")

+----------+--------+-------------------+--------+
|RegionName|    City|               date|   value|
+----------+--------+-------------------+--------+
|     10001|New York|1996-01-31 00:00:00|279614.0|
|     10001|New York|1996-02-29 00:00:00|284673.0|
|     10001|New York|1996-03-31 00:00:00|285884.0|
|     10001|New York|1996-04-30 00:00:00|292432.0|
|     10001|New York|1996-05-31 00:00:00|295610.0|
+----------+--------+-------------------+--------+
only showing top 5 rows




-----Code executed succesfully-----


In [ ]:
sre_prophet.printSchema()

root
 |-- RegionName: string (nullable = true)
 |-- City: string (nullable = true)
 |-- date: timestamp (nullable = true)
 |-- value: double (nullable = true)



In [ ]:
sre_prophet.select(['RegionName']).groupBy('RegionName').agg({'RegionName':'count'}).show()

+----------+-----------------+
|RegionName|count(RegionName)|
+----------+-----------------+
|     10001|              291|
+----------+-----------------+



In [ ]:
sre_prophet.createOrReplaceTempView("value")

In [ ]:
spark.sql("select RegionName, count(*) from value group by RegionName order by RegionName").show()

+----------+--------+
|RegionName|count(1)|
+----------+--------+
|     10001|     291|
+----------+--------+



In [ ]:
sql = 'SELECT RegionName, date as ds, sum(value) as y FROM value GROUP BY RegionName, ds ORDER BY RegionName, ds'

In [ ]:
spark.sql(sql).show()

+----------+-------------------+--------+
|RegionName|                 ds|       y|
+----------+-------------------+--------+
|     10001|1996-01-31 00:00:00|279614.0|
|     10001|1996-02-29 00:00:00|284673.0|
|     10001|1996-03-31 00:00:00|285884.0|
|     10001|1996-04-30 00:00:00|292432.0|
|     10001|1996-05-31 00:00:00|295610.0|
|     10001|1996-06-30 00:00:00|301629.0|
|     10001|1996-07-31 00:00:00|305750.0|
|     10001|1996-08-31 00:00:00|311539.0|
|     10001|1996-09-30 00:00:00|319590.0|
|     10001|1996-10-31 00:00:00|325514.0|
|     10001|1996-11-30 00:00:00|330398.0|
|     10001|1996-12-31 00:00:00|333069.0|
|     10001|1997-01-31 00:00:00|338388.0|
|     10001|1997-02-28 00:00:00|344245.0|
|     10001|1997-03-31 00:00:00|352668.0|
|     10001|1997-04-30 00:00:00|356969.0|
|     10001|1997-05-31 00:00:00|364187.0|
|     10001|1997-06-30 00:00:00|366519.0|
|     10001|1997-07-31 00:00:00|371849.0|
|     10001|1997-08-31 00:00:00|375867.0|
+----------+-------------------+--

In [ ]:
sre_prophet.explain()

== Physical Plan ==
Scan ExistingRDD[RegionName#3176,City#3177,date#3178,value#3179]


In [ ]:
sre_prophet.rdd.getNumPartitions()

2

In [ ]:
RegionName_part = (spark.sql(sql).repartition(spark.sparkContext.defaultParallelism, ['RegionName'])).cache()

In [ ]:
RegionName_part

DataFrame[RegionName: string, ds: timestamp, y: double]

In [ ]:
RegionName_part.explain

<bound method DataFrame.explain of DataFrame[RegionName: string, ds: timestamp, y: double]>

In [ ]:
#Create result schema
result_schema = StructType({
    StructField('ds',TimestampType()),
    StructField('RegionName',StringType()),
    StructField('y',DoubleType()),
    StructField('yhat',DoubleType()),
    StructField('yhat_upper',DoubleType()),
    StructField('yhat_lower',DoubleType()),
})

In [ ]:
from pyspark.sql.functions import pandas_udf, PandasUDFType

@pandas_udf( result_schema, PandasUDFType.GROUPED_MAP )
def forecast_values( RegionName_pd ):
  model = Prophet(interval_width=0.95)
  model.fit( RegionName_pd )
  future_pd = model.make_future_dataframe(
      periods=5,
      freq='w'
      )
  forecast_pd = model.predict ( future_pd )

  f_pd = forecast_pd[ ['ds', 'yhat', 'yhat_upper', 'yhat_lower'] ].set_index('ds')

  st_pd = RegionName_pd[['ds', 'RegionName', 'y']].set_index('ds')

  results_pd = f_pd.join( st_pd, how = 'left')
  results_pd.reset_index(level=0, inplace=True)

  results_pd['RegionName']=  RegionName_pd['RegionName'].iloc[0]

  return results_pd[ ['ds', 'RegionName', 'y', 'yhat','yhat_upper', 'yhat_lower'] ]

print("\n\n\n-----Code executed succesfully-----")




-----Code executed succesfully-----


In [ ]:
results = (
    RegionName_part 
      .groupBy('RegionName')
      .apply(forecast_values)
  )

In [ ]:
results.cache()

DataFrame[ds: timestamp, yhat: double, RegionName: string, yhat_upper: double, y: double, yhat_lower: double]

In [ ]:
results.explain()

== Physical Plan ==
InMemoryTableScan [ds#3281, yhat#3282, RegionName#3283, yhat_upper#3284, y#3285, yhat_lower#3286]
   +- InMemoryRelation [ds#3281, yhat#3282, RegionName#3283, yhat_upper#3284, y#3285, yhat_lower#3286], StorageLevel(disk, memory, deserialized, 1 replicas)
         +- FlatMapGroupsInPandas [RegionName#3176], forecast_values(RegionName#3176, ds#3250, y#3251), [ds#3281, yhat#3282, RegionName#3283, yhat_upper#3284, y#3285, yhat_lower#3286]
            +- *(1) Sort [RegionName#3176 ASC NULLS FIRST], false, 0
               +- InMemoryTableScan [RegionName#3176, RegionName#3176, ds#3250, y#3251]
                     +- InMemoryRelation [RegionName#3176, ds#3250, y#3251], StorageLevel(disk, memory, deserialized, 1 replicas)
                           +- Exchange hashpartitioning(RegionName#3176, 2)
                              +- *(3) Sort [RegionName#3176 ASC NULLS FIRST, ds#3250 ASC NULLS FIRST], true, 0
                                 +- Exchange rangepartitioning(Regi

In [ ]:
results.coalesce

<bound method DataFrame.coalesce of DataFrame[ds: timestamp, yhat: double, RegionName: string, yhat_upper: double, y: double, yhat_lower: double]>

In [ ]:
results.count

<bound method DataFrame.count of DataFrame[ds: timestamp, yhat: double, RegionName: string, yhat_upper: double, y: double, yhat_lower: double]>

In [ ]:
results.show()

Py4JJavaError: ignored